<a href="https://colab.research.google.com/github/Rishit605/Resume-Analysis-NLP/blob/main/NLP_Resume_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install keras_preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.4 MB/s eta 0:00:00


In [2]:
import os
import pathlib
import zipfile
import requests
from pathlib import Path

import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [3]:
! mkdir dataset

In [4]:
cd dataset

/content/dataset


In [5]:
# Fetch a single <1MB file using the raw GitHub URL.
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githubusercontent.com/manishshettym/ResumeRise/master/Data/resume_dataset.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8594k  100 8594k    0     0  13.2M      0 --:--:-- --:--:-- --:--:-- 13.2M


In [6]:
df= pd.read_csv('/content/dataset/resume_dataset.csv')
df

,ID,Category,Resume
0,1,HR,"b'John H. Smith, P.H.R.\n800-991-5187 | PO Box..."
1,2,HR,b'Name Surname\nAddress\nMobile No/Email\nPERS...
2,3,HR,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...
3,4,HR,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...
4,5,HR,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert..."
...,...,...,...
1214,1215,Aviation,"b""Free Flight Attendant Resume\nDarlene Flint\..."
1215,1216,Aviation,b'Corporate Flight Attendant Resume\nCAITLIN F...
1216,1217,Aviation,"b'MAJOR CONRAD A. PREEDOM\n2354 Fairchild Dr.,..."
1217,1218,Aviation,b'STACY SAMPLE\n\n702 800-0000 cell\n\n0000@em...


In [7]:
df["Resume"][0]

"b'John H. Smith, P.H.R.\\n800-991-5187 | PO Box 1673 | Callahan, FL 32011 | info@greatresumesfast.com\\n\\nApproachable innovator with a passion for Human Resources.\\n\\nSENIOR HUMAN RESOURCES PROFESSIONAL\\nPersonable, analytical, flexible Senior HR Professional with multifaceted expertise. Seasoned Benefits Administrator with\\nextensive experience working with highly paid professionals in client-relationship-based settings. Dynamic team leader\\ncapable of analyzing alternatives and identifying tough choices while communicating the total value of benefit and\\ncompensation packages to senior level executives and employees.\\n\\nCORE COMPETENCIES\\nBenefits Administration \\xe2\\x80\\x93 Customer Service \\xe2\\x80\\x93 Cost Control \\xe2\\x80\\x93 Recruiting \\xe2\\x80\\x93 Acquisition Management \\xe2\\x80\\x93 Compliance Reporting\\nRetention \\xe2\\x80\\x93 Professional Services \\xe2\\x80\\x93 Domestic & International Benefits \\xe2\\x80\\x93 Collaboration \\xe2\\x80\\x93 Adap

In [8]:
df2 = df.copy(deep=True)
df2

,ID,Category,Resume
0,1,HR,"b'John H. Smith, P.H.R.\n800-991-5187 | PO Box..."
1,2,HR,b'Name Surname\nAddress\nMobile No/Email\nPERS...
2,3,HR,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...
3,4,HR,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...
4,5,HR,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert..."
...,...,...,...
1214,1215,Aviation,"b""Free Flight Attendant Resume\nDarlene Flint\..."
1215,1216,Aviation,b'Corporate Flight Attendant Resume\nCAITLIN F...
1216,1217,Aviation,"b'MAJOR CONRAD A. PREEDOM\n2354 Fairchild Dr.,..."
1217,1218,Aviation,b'STACY SAMPLE\n\n702 800-0000 cell\n\n0000@em...


In [9]:
classes = list(df['Category'].unique())
print(classes, len(classes))

['HR', 'Designing', 'Managment', 'Information Technology', 'Education', 'Advocate', 'Business Development', 'Health & Fitness', 'Agricultural', 'BPO', 'Sales', 'Consultant', 'Digital Media', 'Automobile', 'Food & Beverages', 'Finance', 'Apparel', 'Engineering', 'Accountant', 'Building & Construction', 'Architects', 'Public Relations', 'Banking', 'Arts', 'Aviation'] 25


In [10]:
classes = list(df['Category'].unique())
print(classes, len(classes))

class_idx = {key: value for value, key in enumerate(classes)}
print(classes)

['HR', 'Designing', 'Managment', 'Information Technology', 'Education', 'Advocate', 'Business Development', 'Health & Fitness', 'Agricultural', 'BPO', 'Sales', 'Consultant', 'Digital Media', 'Automobile', 'Food & Beverages', 'Finance', 'Apparel', 'Engineering', 'Accountant', 'Building & Construction', 'Architects', 'Public Relations', 'Banking', 'Arts', 'Aviation']


# Data Cleaning

In [11]:
import string
def rem_punc(s):
    punc = string.punctuation
    return [i for i in s if i not in punc]

def rem_sw(s):
    sw = set(stopwords)
    return [i for i in s if i not in sw]

In [12]:
import re

# def normalize(data):
#     normalized = []
#     for i in data:
#         # get rid of urls
#         i = re.sub('https?://\S+|www\.\S+', '', i)
#         # get rid of non words and extra spaces
#         i = re.sub('\\W', ' ', i)
#         i = re.sub('\n', '', i)
#         i = re.sub(' +', ' ', i)
#         i = re.sub('^ ', '', i)
#         i = re.sub(' $', '', i)
#         i = i.lower()
#         normalized.append(i)
#     return normalized

# import re

import re

def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'www\S+', '', text)
    # Remove punctuations
    text = re.sub(r'[^\w\s]', '', text)
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text)
    # Convert to lowercase
    text = text.lower()
    return text


In [13]:
df2['Resume']

0       b'John H. Smith, P.H.R.\n800-991-5187 | PO Box...
1       b'Name Surname\nAddress\nMobile No/Email\nPERS...
2       b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...
3       b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...
4       b"HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert...
                              ...                        
1214    b"Free Flight Attendant Resume\nDarlene Flint\...
1215    b'Corporate Flight Attendant Resume\nCAITLIN F...
1216    b'MAJOR CONRAD A. PREEDOM\n2354 Fairchild Dr.,...
1217    b'STACY SAMPLE\n\n702 800-0000 cell\n\n0000@em...
1218    b'Entry Level Resume Guide\n\nThis packet is i...
Name: Resume, Length: 1219, dtype: object

In [14]:
df2

,ID,Category,Resume
0,1,HR,"b'John H. Smith, P.H.R.\n800-991-5187 | PO Box..."
1,2,HR,b'Name Surname\nAddress\nMobile No/Email\nPERS...
2,3,HR,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...
3,4,HR,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...
4,5,HR,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert..."
...,...,...,...
1214,1215,Aviation,"b""Free Flight Attendant Resume\nDarlene Flint\..."
1215,1216,Aviation,b'Corporate Flight Attendant Resume\nCAITLIN F...
1216,1217,Aviation,"b'MAJOR CONRAD A. PREEDOM\n2354 Fairchild Dr.,..."
1217,1218,Aviation,b'STACY SAMPLE\n\n702 800-0000 cell\n\n0000@em...


In [15]:
df2['normalized_text'] = df2['Resume'].apply(preprocess_text)

In [16]:
df2['normalized_text']

0       bjohn h smith phrn8009915187 po box 1673 calla...
1       bname surnamenaddressnmobile noemailnpersonal ...
2       banthony brownnhr assistantnareas of expertise...
3       b idncareer objectivento pursue a growth orien...
4       bhuman resources directornxefx82xb7expert in o...
                              ...                        
1214    bfree flight attendant resumendarlene flintn80...
1215    bcorporate flight attendant resumencaitlin fla...
1216    bmajor conrad a preedomn2354 fairchild dr suit...
1217    bstacy samplenn702 8000000 cellnn0000emailcomn...
1218    bentry level resume guidennthis packet is inte...
Name: normalized_text, Length: 1219, dtype: object

In [17]:
df2

,ID,Category,Resume,normalized_text
0,1,HR,"b'John H. Smith, P.H.R.\n800-991-5187 | PO Box...",bjohn h smith phrn8009915187 po box 1673 calla...
1,2,HR,b'Name Surname\nAddress\nMobile No/Email\nPERS...,bname surnamenaddressnmobile noemailnpersonal ...
2,3,HR,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...,banthony brownnhr assistantnareas of expertise...
3,4,HR,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...,b idncareer objectivento pursue a growth orien...
4,5,HR,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert...",bhuman resources directornxefx82xb7expert in o...
...,...,...,...,...
1214,1215,Aviation,"b""Free Flight Attendant Resume\nDarlene Flint\...",bfree flight attendant resumendarlene flintn80...
1215,1216,Aviation,b'Corporate Flight Attendant Resume\nCAITLIN F...,bcorporate flight attendant resumencaitlin fla...
1216,1217,Aviation,"b'MAJOR CONRAD A. PREEDOM\n2354 Fairchild Dr.,...",bmajor conrad a preedomn2354 fairchild dr suit...
1217,1218,Aviation,b'STACY SAMPLE\n\n702 800-0000 cell\n\n0000@em...,bstacy samplenn702 8000000 cellnn0000emailcomn...


In [18]:
length = df['Resume'].shape
length

(1219,)

In [19]:
#eval_res = Resumes
j=0
i=0
l=[]
for i in range(length[0]):
    try:
        df2['Resume'][i] = eval(df2['Resume'][i]).decode()
    except:
        l.append(i)
        pass

<ipython-input-19-391ffd89cddb>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Resume'][i] = eval(df2['Resume'][i]).decode()


In [20]:
l

[34, 347, 489, 501, 742, 900, 915]

In [21]:
df2["res_new"] = df['Resume']
df2 = df2.drop(l,axis=0)
#print(df[30:40])
df3 = df2.reset_index(drop = True)

In [22]:
df3

,ID,Category,Resume,normalized_text,res_new
0,1,HR,"John H. Smith, P.H.R.\n800-991-5187 | PO Box 1...",bjohn h smith phrn8009915187 po box 1673 calla...,"b'John H. Smith, P.H.R.\n800-991-5187 | PO Box..."
1,2,HR,Name Surname\nAddress\nMobile No/Email\nPERSON...,bname surnamenaddressnmobile noemailnpersonal ...,b'Name Surname\nAddress\nMobile No/Email\nPERS...
2,3,HR,Anthony Brown\nHR Assistant\nAREAS OF EXPERTIS...,banthony brownnhr assistantnareas of expertise...,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...
3,4,HR,www.downloadmela.com\nSatheesh\nEMAIL ID:\nCar...,b idncareer objectivento pursue a growth orien...,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...
4,5,HR,HUMAN RESOURCES DIRECTOR\nExpert in organizat...,bhuman resources directornxefx82xb7expert in o...,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert..."
...,...,...,...,...,...
1207,1215,Aviation,Free Flight Attendant Resume\nDarlene Flint\n8...,bfree flight attendant resumendarlene flintn80...,"b""Free Flight Attendant Resume\nDarlene Flint\..."
1208,1216,Aviation,Corporate Flight Attendant Resume\nCAITLIN FLA...,bcorporate flight attendant resumencaitlin fla...,b'Corporate Flight Attendant Resume\nCAITLIN F...
1209,1217,Aviation,"MAJOR CONRAD A. PREEDOM\n2354 Fairchild Dr., S...",bmajor conrad a preedomn2354 fairchild dr suit...,"b'MAJOR CONRAD A. PREEDOM\n2354 Fairchild Dr.,..."
1210,1218,Aviation,STACY SAMPLE\n\n702 800-0000 cell\n\n0000@emai...,bstacy samplenn702 8000000 cellnn0000emailcomn...,b'STACY SAMPLE\n\n702 800-0000 cell\n\n0000@em...


In [23]:
df3['normalized_text'] = df3['Resume'].apply(preprocess_text)

In [24]:
df3

,ID,Category,Resume,normalized_text,res_new
0,1,HR,"John H. Smith, P.H.R.\n800-991-5187 | PO Box 1...",john h smith phr 8009915187 po box 1673 callah...,"b'John H. Smith, P.H.R.\n800-991-5187 | PO Box..."
1,2,HR,Name Surname\nAddress\nMobile No/Email\nPERSON...,name surname address mobile noemail personal p...,b'Name Surname\nAddress\nMobile No/Email\nPERS...
2,3,HR,Anthony Brown\nHR Assistant\nAREAS OF EXPERTIS...,anthony brown hr assistant areas of expertise ...,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...
3,4,HR,www.downloadmela.com\nSatheesh\nEMAIL ID:\nCar...,satheesh email id career objective to pursue ...,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...
4,5,HR,HUMAN RESOURCES DIRECTOR\nExpert in organizat...,human resources director expert in organizatio...,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert..."
...,...,...,...,...,...
1207,1215,Aviation,Free Flight Attendant Resume\nDarlene Flint\n8...,free flight attendant resume darlene flint 809...,"b""Free Flight Attendant Resume\nDarlene Flint\..."
1208,1216,Aviation,Corporate Flight Attendant Resume\nCAITLIN FLA...,corporate flight attendant resume caitlin flan...,b'Corporate Flight Attendant Resume\nCAITLIN F...
1209,1217,Aviation,"MAJOR CONRAD A. PREEDOM\n2354 Fairchild Dr., S...",major conrad a preedom 2354 fairchild dr suite...,"b'MAJOR CONRAD A. PREEDOM\n2354 Fairchild Dr.,..."
1210,1218,Aviation,STACY SAMPLE\n\n702 800-0000 cell\n\n0000@emai...,stacy sample 702 8000000 cell 0000emailcom qua...,b'STACY SAMPLE\n\n702 800-0000 cell\n\n0000@em...


In [25]:
df3

,ID,Category,Resume,normalized_text,res_new
0,1,HR,"John H. Smith, P.H.R.\n800-991-5187 | PO Box 1...",john h smith phr 8009915187 po box 1673 callah...,"b'John H. Smith, P.H.R.\n800-991-5187 | PO Box..."
1,2,HR,Name Surname\nAddress\nMobile No/Email\nPERSON...,name surname address mobile noemail personal p...,b'Name Surname\nAddress\nMobile No/Email\nPERS...
2,3,HR,Anthony Brown\nHR Assistant\nAREAS OF EXPERTIS...,anthony brown hr assistant areas of expertise ...,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...
3,4,HR,www.downloadmela.com\nSatheesh\nEMAIL ID:\nCar...,satheesh email id career objective to pursue ...,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...
4,5,HR,HUMAN RESOURCES DIRECTOR\nExpert in organizat...,human resources director expert in organizatio...,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert..."
...,...,...,...,...,...
1207,1215,Aviation,Free Flight Attendant Resume\nDarlene Flint\n8...,free flight attendant resume darlene flint 809...,"b""Free Flight Attendant Resume\nDarlene Flint\..."
1208,1216,Aviation,Corporate Flight Attendant Resume\nCAITLIN FLA...,corporate flight attendant resume caitlin flan...,b'Corporate Flight Attendant Resume\nCAITLIN F...
1209,1217,Aviation,"MAJOR CONRAD A. PREEDOM\n2354 Fairchild Dr., S...",major conrad a preedom 2354 fairchild dr suite...,"b'MAJOR CONRAD A. PREEDOM\n2354 Fairchild Dr.,..."
1210,1218,Aviation,STACY SAMPLE\n\n702 800-0000 cell\n\n0000@emai...,stacy sample 702 8000000 cell 0000emailcom qua...,b'STACY SAMPLE\n\n702 800-0000 cell\n\n0000@em...


In [26]:
# texts = 

# # Step 1: Convert text to lowercase
# # texts = texts.lower()

# # Step 2: Split text into words
# words = texts.split()

# # Step 3: Create dictionary to store word frequency counts
# word_freq = {}

# # Step 4: Loop through words and update dictionary
# for word in words:
#     if word in word_freq:
#         word_freq[word] += 1
#     else:
#         word_freq[word] = 1

# # Print the word frequency counts
# for word, count in word_freq.items():
#     print(f"{word}: {count}")


In [ ]:
df4 = df3[['Category', 'normalized_text']]
df4

In [28]:
# initialize a LabelEncoder object
label_encoder = LabelEncoder()

# encode the categorical labels as numerical values
df4['Label'] = label_encoder.fit_transform(df4['Category'])

<ipython-input-28-ca7d075163ca>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['Label'] = label_encoder.fit_transform(df4['Category'])


In [29]:
df4

,Category,normalized_text,Label
0,HR,john h smith phr 8009915187 po box 1673 callah...,19
1,HR,name surname address mobile noemail personal p...,19
2,HR,anthony brown hr assistant areas of expertise ...,19
3,HR,satheesh email id career objective to pursue ...,19
4,HR,human resources director expert in organizatio...,19
...,...,...,...
1207,Aviation,free flight attendant resume darlene flint 809...,7
1208,Aviation,corporate flight attendant resume caitlin flan...,7
1209,Aviation,major conrad a preedom 2354 fairchild dr suite...,7
1210,Aviation,stacy sample 702 8000000 cell 0000emailcom qua...,7


In [30]:
df4.sort_values(by='Label', inplace=True)

<ipython-input-30-f6b65e4cd1cf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.sort_values(by='Label', inplace=True)


In [37]:
print(f"{df4} DataFrame with {df4['Label']} \nhas {df4['Label'].unique()} values")

        Category                                    normalized_text  Label
1000  Accountant  core competencies administrative tasks data an...      0
1037  Accountant  chartered accountant internship resume craig m...      0
1038  Accountant  bank accountant resume bank accountant resume ...      0
1039  Accountant  gary white account manager areas of expertise ...      0
1040  Accountant  professional tax accountant resume john doe 1c...      0
...          ...                                                ...    ...
625        Sales                                                        24
626        Sales  g e o r g e h e r n a nd e z home 617 3622121 ...     24
627        Sales  pharmaceutical sales representative awardwinni...     24
659        Sales  sales clerk resume template joye thompson 1234...     24
605        Sales  george redmond 10 sanford drive andover me 042...     24

[1212 rows x 3 columns] DataFrame with 1000     0
1037     0
1038     0
1039     0
1040     0
     

In [ ]:
train_df = df4['normalized_text'].tolist()
train_df

In [39]:
### PARAMETERS ### 

vocab_size = 10000
embed_dim = 64
max_length = 120
trunc_type = 'pre'
oov_tok = "<OOV>"

In [40]:
token = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
token.fit_on_texts(df4['normalized_text'].values)

word_idx = token.word_index

In [41]:
train_seq = token.texts_to_sequences(df4['normalized_text'].values)
train_padded = pad_sequences(train_seq, maxlen=max_length, truncating=trunc_type)

# test_seq = token.texts_to_sequences(test_df['text'].values)
# test_padded = pad_sequences(test_seq, maxlen=max_length)


In [66]:
train_padded

array([[2462,  355,  181, ...,   71,    2,  169],
       [ 225, 1660,   34, ...,  225,   63, 1510],
       [ 159,    7, 2195, ...,  494,  509,    1],
       ...,
       [  18, 5182, 4294, ..., 1878,   32,  147],
       [   3,  367,    2, ...,    4, 2503, 8285],
       [  58, 3968,    2, ...,   87,    1,  530]], dtype=int32)

In [68]:
model = Sequential([
    layers.Embedding(vocab_size, embed_dim, input_length=max_length),
    layers.Conv1D(128, 8, activation='relu'),
    layers.GlobalMaxPooling1D(),
    
    layers.Dense(48, activation='relu'),
    layers.Dense(25, activation='softmax'),
])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 120, 64)           640000    
                                                                 
 conv1d_4 (Conv1D)           (None, 113, 128)          65664     
                                                                 
 global_max_pooling1d_4 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_8 (Dense)             (None, 48)                6192      
                                                                 
 dense_9 (Dense)             (None, 25)                1225      
                                                                 
Total params: 713,081
Trainable params: 713,081
Non-trainable params: 0
________________________________________________

In [69]:
model.output_shape

(None, 25)

In [70]:
# opt = keras.optimizers.SGD(learning_rate=1e-3, weight_decay=1e-2, momentum=0.9)
opt = keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])

In [71]:
y_train = pd.get_dummies(df4['Category']).values
y_train

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]], dtype=uint8)

In [72]:
df4['Label'].values.shape

(1212,)

In [73]:
history = model.fit(train_padded, y_train, #df4['Label'].values
                   epochs=10, batch_size=32) #test_df['label']. values))

Epoch 1/10
38/38 [==============================] - 4s 48ms/step - loss: 3.1682 - accuracy: 0.0800 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 2/10
38/38 [==============================] - 2s 46ms/step - loss: 3.0010 - accuracy: 0.1320 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 3/10
38/38 [==============================] - 2s 53ms/step - loss: 2.8440 - accuracy: 0.2327 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 4/10
38/38 [==============================] - 3s 73ms/step - loss: 2.6026 - accuracy: 0.3276 - precision: 1.0000 - recall: 0.0083
Epoch 5/10
38/38 [==============================] - 2s 46ms/step - loss: 2.1753 - accuracy: 0.4917 - precision: 0.9500 - recall: 0.0470
Epoch 6/10
38/38 [==============================] - 2s 48ms/step - loss: 1.6382 - accuracy: 0.6419 - precision: 0.9645 - recall: 0.2244
Epoch 7/10
38/38 [==============================] - 2s 46ms/step - loss: 1.1305 - accuracy: 0.8036 - precision: 0.9755 - recall: 0.4266
Epoch 8/10
38/38 [======

# Predicting

In [74]:
! pip install pdfminer.six==20181108

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.1 MB/s eta 0:00:00


In [75]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

def convertPDFtoText(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    fp.close()
    device.close()
    string = retstr.getvalue()
    retstr.close()
    return string

In [76]:
test_resume = convertPDFtoText('/content/sample_input.pdf')
print(test_resume)

Dushyant Bhatt

BI / Big Data/ Azure

Hyderabad-Deccan, Telangana, Telangana - Email me on Indeed: indeed.com/r/Dushyant-
Bhatt/140749dace5dc26f

• 10+ years of Experience in Designing, Development, Administration, Analysis, Management in
the Business Intelligence Data warehousing, Client Server Technologies, Web-based Applications,
cloud solutions and Databases.
• Data warehouse: Data analysis, star/ snow flake schema data modeling and design specific to
data warehousing and business intelligence environment.
• Database: Experience in database designing, scalability, back-up and recovery, writing and
optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. 
• Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, Stream
Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake analytics(U-SQL).
• Big Data: Worked Azure data lake store/analytics for big data processing and Azure data factory
to schedule

In [77]:
from nltk.probability import FreqDist
from string import punctuation
import math
def summarize(doc,words):
    score={}
    fd = FreqDist(words)
    for i,t in enumerate(doc):
        score[i] = 0
        for j in nltk.word_tokenize(t):
            if j in fd:
                score[i]+=fd[j]
    
    r = sorted(list(score.items()),key=lambda x:x[1],reverse=True)[:math.floor(0.60*len(doc))]
    r.sort(key=lambda x:x[0])
    l = [doc[i[0]] for i in r]
    return "\n\n".join(l)

In [78]:
def preprocess(eval_res):
    try:
        eval_res = eval(eval_res).decode()
    except:
        pass
    eval_res = eval_res.encode("ASCII","ignore").decode()
    length = len(eval_res)
    eval_res = " ".join(eval_res.split("\n"))
    token = rem_sw(nltk.word_tokenize(eval_res)) #Removing punctaution later since we need punctaution for sentence tokenization
    eval_res = " ".join(token).lower()
    return eval_res

In [79]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [80]:
resume = preprocess(test_resume)#remove stop words etc
sent = nltk.sent_tokenize(test_resume)
puncu = punctuation
word_token = nltk.word_tokenize(test_resume)#tokenize preprocessed text for scoring

print(summarize(sent,test_resume))

Dushyant Bhatt

BI / Big Data/ Azure

Hyderabad-Deccan, Telangana, Telangana - Email me on Indeed: indeed.com/r/Dushyant-
Bhatt/140749dace5dc26f

• 10+ years of Experience in Designing, Development, Administration, Analysis, Management in
the Business Intelligence Data warehousing, Client Server Technologies, Web-based Applications,
cloud solutions and Databases.

• Data warehouse: Data analysis, star/ snow flake schema data modeling and design specific to
data warehousing and business intelligence environment.

• Database: Experience in database designing, scalability, back-up and recovery, writing and
optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes.

• Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, Stream
Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake analytics(U-SQL).

• BI:
o  ETL:  Designed  and  developed  ETL  solution  in  SSIS.

Experience  in  Logging,  Error  handli

In [81]:
word_token

['Dushyant',
 'Bhatt',
 'BI',
 '/',
 'Big',
 'Data/',
 'Azure',
 'Hyderabad-Deccan',
 ',',
 'Telangana',
 ',',
 'Telangana',
 '-',
 'Email',
 'me',
 'on',
 'Indeed',
 ':',
 'indeed.com/r/Dushyant-',
 'Bhatt/140749dace5dc26f',
 '•',
 '10+',
 'years',
 'of',
 'Experience',
 'in',
 'Designing',
 ',',
 'Development',
 ',',
 'Administration',
 ',',
 'Analysis',
 ',',
 'Management',
 'in',
 'the',
 'Business',
 'Intelligence',
 'Data',
 'warehousing',
 ',',
 'Client',
 'Server',
 'Technologies',
 ',',
 'Web-based',
 'Applications',
 ',',
 'cloud',
 'solutions',
 'and',
 'Databases',
 '.',
 '•',
 'Data',
 'warehouse',
 ':',
 'Data',
 'analysis',
 ',',
 'star/',
 'snow',
 'flake',
 'schema',
 'data',
 'modeling',
 'and',
 'design',
 'specific',
 'to',
 'data',
 'warehousing',
 'and',
 'business',
 'intelligence',
 'environment',
 '.',
 '•',
 'Database',
 ':',
 'Experience',
 'in',
 'database',
 'designing',
 ',',
 'scalability',
 ',',
 'back-up',
 'and',
 'recovery',
 ',',
 'writing',
 'and',


In [82]:
def tok(text):
  token = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
  token.fit_on_texts(text)

  word_idx = token.word_index

  train_seq = token.texts_to_sequences(text)
  train_pad= pad_sequences(train_seq, maxlen=max_length, truncating=trunc_type)

  return train_pad

array([[ 0,  0,  0, ...,  0,  0, 10],
       [ 0,  0,  0, ...,  0,  0, 14],
       [ 0,  0,  0, ...,  0,  0,  4],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0, 34]], dtype=int32)

In [83]:
pred = model.predict(tok(test_resume))

200/200 [==============================] - 2s 11ms/step


In [84]:
predicted = np.argmax(pred, axis=-1)[0]

In [85]:
predicted

16

In [89]:
df4['Label'][16]

19

In [91]:
classes = list(df4['Category'].unique())
print(classes, len(classes))

class_id = {key: value for value, key in enumerate(classes)}
print(class_id)

['Accountant', 'Advocate', 'Agricultural', 'Apparel', 'Architects', 'Arts', 'Automobile', 'Aviation', 'BPO', 'Banking', 'Building & Construction', 'Business Development', 'Consultant', 'Designing', 'Digital Media', 'Education', 'Engineering', 'Finance', 'Food & Beverages', 'HR', 'Health & Fitness', 'Information Technology', 'Managment', 'Public Relations', 'Sales'] 25
{'Accountant': 0, 'Advocate': 1, 'Agricultural': 2, 'Apparel': 3, 'Architects': 4, 'Arts': 5, 'Automobile': 6, 'Aviation': 7, 'BPO': 8, 'Banking': 9, 'Building & Construction': 10, 'Business Development': 11, 'Consultant': 12, 'Designing': 13, 'Digital Media': 14, 'Education': 15, 'Engineering': 16, 'Finance': 17, 'Food & Beverages': 18, 'HR': 19, 'Health & Fitness': 20, 'Information Technology': 21, 'Managment': 22, 'Public Relations': 23, 'Sales': 24}
